In [ ]:
# ===================================================================
# PHASE 3: ENTITY STRUCTURING & RELATIONSHIP EXTRACTION PIPELINE
# Production-Level Code with Error Handling and Comprehensive Logging
# ===================================================================
# Purpose: Transform flat entity strings into structured job postings
#          with proper relationships and validation
# ===================================================================

from __future__ import annotations
import os
import json
import re
import logging
from datetime import datetime, timedelta
from typing import List, Dict, Set, Any, Optional, Tuple
from collections import defaultdict, Counter
from dataclasses import dataclass, field, asdict
import pandas as pd
import numpy as np

# ===================================================================
# LOGGING CONFIGURATION
# ===================================================================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("entity_structuring.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("EntityStructuring")

# ===================================================================
# DATA CLASSES FOR STRUCTURED ENTITIES
# ===================================================================

@dataclass
class Company:
    """Structured company entity with normalization."""
    name: str
    canonical_name: str
    confidence: float = 0.0
    
    def __post_init__(self):
        """Validate and normalize company data."""
        if not self.name:
            raise ValueError("Company name cannot be empty")
        self.canonical_name = self.canonical_name or self._normalize_name()
        self.confidence = max(0.0, min(1.0, self.confidence))
    
    def _normalize_name(self) -> str:
        """Normalize company name to canonical form."""
        name = self.name.upper()
        # Remove common suffixes
        suffixes = ['PVT LTD', 'PVT. LTD.', 'PRIVATE LIMITED', 'LIMITED', 
                   'LTD', 'INC', 'CORP', 'CORPORATION', 'LLC']
        for suffix in suffixes:
            name = name.replace(suffix, '').strip()
        return name.strip()


@dataclass
class Position:
    """Structured job position entity."""
    title: str
    level: str = "Not Specified"  # Junior, Mid, Senior, Lead
    confidence: float = 0.0
    
    def __post_init__(self):
        """Validate and normalize position data."""
        if not self.title:
            raise ValueError("Position title cannot be empty")
        self.title = self.title.title()  # Capitalize properly
        self.level = self._determine_level()
        self.confidence = max(0.0, min(1.0, self.confidence))
    
    def _determine_level(self) -> str:
        """Determine seniority level from title."""
        title_lower = self.title.lower()
        if any(word in title_lower for word in ['senior', 'lead', 'principal', 'staff']):
            return "Senior"
        elif any(word in title_lower for word in ['junior', 'associate', 'entry']):
            return "Junior"
        elif any(word in title_lower for word in ['intern', 'trainee']):
            return "Intern"
        elif any(word in title_lower for word in ['manager', 'head', 'director']):
            return "Manager"
        return "Mid"


@dataclass
class Requirements:
    """Job requirements including skills, education, experience."""
    skills: List[str] = field(default_factory=list)
    education: List[str] = field(default_factory=list)
    experience_min: int = 0
    experience_max: int = 0
    experience_type: str = "Not Specified"
    
    def __post_init__(self):
        """Validate requirements."""
        self.skills = [s.strip().title() for s in self.skills if s.strip()]
        self.education = [e.strip().upper() for e in self.education if e.strip()]
        self.experience_type = self._determine_experience_type()
    
    def _determine_experience_type(self) -> str:
        """Determine experience category."""
        if self.experience_max == 0:
            return "Fresher"
        elif self.experience_max <= 2:
            return "Entry Level"
        elif self.experience_max <= 5:
            return "Mid Level"
        else:
            return "Senior Level"


@dataclass
class Location:
    """Structured location entity."""
    city: str
    state: str = "Not Specified"
    work_mode: str = "On-site"  # On-site, Remote, Hybrid
    confidence: float = 0.0
    
    def __post_init__(self):
        """Validate and normalize location."""
        if not self.city:
            raise ValueError("City cannot be empty")
        self.city = self._normalize_city()
        self.work_mode = self._determine_work_mode()
        self.confidence = max(0.0, min(1.0, self.confidence))
    
    def _normalize_city(self) -> str:
        """Normalize city name to canonical form."""
        # City name mappings
        city_map = {
            'bangalore': 'Bangalore',
            'bengaluru': 'Bangalore',
            'blr': 'Bangalore',
            'mumbai': 'Mumbai',
            'bombay': 'Mumbai',
            'delhi': 'Delhi',
            'new delhi': 'Delhi',
            'ncr': 'Delhi NCR',
            'gurgaon': 'Gurgaon',
            'gurugram': 'Gurgaon',
            'hyderabad': 'Hyderabad',
            'pune': 'Pune',
            'chennai': 'Chennai',
            'kolkata': 'Kolkata',
            'calcutta': 'Kolkata'
        }
        return city_map.get(self.city.lower(), self.city.title())
    
    def _determine_work_mode(self) -> str:
        """Determine work mode from city name."""
        city_lower = self.city.lower()
        if any(word in city_lower for word in ['remote', 'wfh', 'work from home', 'anywhere']):
            return "Remote"
        elif 'hybrid' in city_lower:
            return "Hybrid"
        return "On-site"


@dataclass
class Compensation:
    """Structured compensation information."""
    salary_min: int = 0
    salary_max: int = 0
    currency: str = "INR"
    period: str = "annual"  # annual, monthly
    raw_text: str = ""
    confidence: float = 0.0
    
    def __post_init__(self):
        """Validate compensation data."""
        if self.salary_min > self.salary_max and self.salary_max > 0:
            self.salary_min, self.salary_max = self.salary_max, self.salary_min
        self.confidence = max(0.0, min(1.0, self.confidence))


@dataclass
class Application:
    """Application details including deadline."""
    deadline: Optional[str] = None
    apply_link: Optional[str] = None
    contact_email: Optional[str] = None
    
    def __post_init__(self):
        """Validate application data."""
        if self.deadline:
            self.deadline = self._normalize_deadline()
    
    def _normalize_deadline(self) -> Optional[str]:
        """Normalize deadline to ISO format."""
        try:
            # Try parsing common formats
            for fmt in ['%Y-%m-%d', '%d-%m-%Y', '%d/%m/%Y', '%d/%m/%y']:
                try:
                    dt = datetime.strptime(self.deadline, fmt)
                    return dt.strftime('%Y-%m-%d')
                except:
                    continue
            return self.deadline
        except:
            return None


@dataclass
class JobPosting:
    """Complete structured job posting."""
    job_id: str
    email_id: str
    company: Company
    position: Position
    requirements: Requirements
    location: Location
    compensation: Compensation
    application: Application
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for JSON serialization."""
        return {
            'job_id': self.job_id,
            'email_id': self.email_id,
            'company': asdict(self.company),
            'position': asdict(self.position),
            'requirements': asdict(self.requirements),
            'location': asdict(self.location),
            'compensation': asdict(self.compensation),
            'application': asdict(self.application),
            'metadata': self.metadata
        }
    
    def calculate_completeness(self) -> float:
        """Calculate how complete this job posting is (0-1)."""
        scores = []
        scores.append(1.0 if self.company.name else 0.0)
        scores.append(1.0 if self.position.title else 0.0)
        scores.append(0.8 if self.requirements.skills else 0.0)
        scores.append(0.6 if self.location.city else 0.0)
        scores.append(0.5 if self.compensation.salary_max > 0 else 0.0)
        scores.append(0.3 if self.requirements.education else 0.0)
        scores.append(0.2 if self.application.deadline else 0.0)
        return sum(scores) / len(scores)


# ===================================================================
# ENTITY NORMALIZER
# ===================================================================

class EntityNormalizer:
    """
    Normalize and standardize extracted entities.
    Handles variations, aliases, and canonical forms.
    """
    
    def __init__(self):
        """Initialize normalizer with mapping dictionaries."""
        self.logger = logging.getLogger("EntityNormalizer")
        
        # Skill normalization map
        self.skill_map = {
            'js': 'javascript',
            'ts': 'typescript',
            'py': 'python',
            'reactjs': 'react',
            'nodejs': 'node.js',
            'ml': 'machine learning',
            'ai': 'artificial intelligence',
            'dl': 'deep learning',
            'nlp': 'natural language processing',
            'cv': 'computer vision',
            'k8s': 'kubernetes',
            'tf': 'tensorflow',
            'scikit': 'scikit-learn'
        }
        
        # Degree normalization map
        self.degree_map = {
            'btech': 'B.Tech',
            'b.tech': 'B.Tech',
            'be': 'B.E',
            'b.e': 'B.E',
            'mtech': 'M.Tech',
            'm.tech': 'M.Tech',
            'me': 'M.E',
            'm.e': 'M.E',
            'bca': 'BCA',
            'mca': 'MCA',
            'bsc': 'B.Sc',
            'b.sc': 'B.Sc',
            'msc': 'M.Sc',
            'm.sc': 'M.Sc'
        }
    
    def normalize_skill(self, skill: str) -> str:
        """
        Normalize skill to canonical form.
        
        Args:
            skill: Raw skill string
            
        Returns:
            Normalized skill name
        """
        skill_lower = skill.lower().strip()
        return self.skill_map.get(skill_lower, skill_lower)
    
    def normalize_degree(self, degree: str) -> str:
        """
        Normalize degree to canonical form.
        
        Args:
            degree: Raw degree string
            
        Returns:
            Normalized degree name
        """
        degree_lower = degree.lower().strip()
        return self.degree_map.get(degree_lower, degree.upper())
    
    def normalize_skills_list(self, skills: List[str]) -> List[str]:
        """Normalize a list of skills."""
        normalized = []
        seen = set()
        for skill in skills:
            norm_skill = self.normalize_skill(skill)
            if norm_skill not in seen:
                normalized.append(norm_skill)
                seen.add(norm_skill)
        return normalized
    
    def normalize_degrees_list(self, degrees: List[str]) -> List[str]:
        """Normalize a list of degrees."""
        normalized = []
        seen = set()
        for degree in degrees:
            norm_degree = self.normalize_degree(degree)
            if norm_degree not in seen:
                normalized.append(norm_degree)
                seen.add(norm_degree)
        return normalized


# ===================================================================
# SALARY PARSER
# ===================================================================

class SalaryParser:
    """
    Parse and normalize salary information from text.
    Handles various formats: LPA, monthly, ranges, etc.
    """
    
    def __init__(self):
        """Initialize salary parser with regex patterns."""
        self.logger = logging.getLogger("SalaryParser")
        
        # Salary patterns
        self.patterns = [
            # Pattern: "5-8 LPA" or "5 to 8 LPA"
            (r'(\d+(?:\.\d+)?)\s*(?:-|to)\s*(\d+(?:\.\d+)?)\s*(?:lpa|lakhs?\s+per\s+annum)', 'lpa_range'),
            # Pattern: "5.5 LPA"
            (r'(\d+(?:\.\d+)?)\s*(?:lpa|lakhs?\s+per\s+annum)', 'lpa_single'),
            # Pattern: "20k per month" or "20000 per month"
            (r'(\d+)k?\s*(?:per\s+month|pm|/month)', 'monthly'),
            # Pattern: "CTC: 5 LPA"
            (r'ctc\s*:?\s*(\d+(?:\.\d+)?)\s*(?:lpa|lakhs?)', 'ctc'),
            # Pattern: "Package: 5-8 lakhs"
            (r'package\s*:?\s*(\d+(?:\.\d+)?)\s*(?:-|to)\s*(\d+(?:\.\d+)?)\s*lakhs?', 'package_range')
        ]
    
    def parse(self, text: str) -> Optional[Compensation]:
        """
        Parse salary information from text.
        
        Args:
            text: Input text containing salary info
            
        Returns:
            Compensation object or None
        """
        if not text or not isinstance(text, str):
            return None
        
        text_lower = text.lower()
        
        try:
            for pattern, pattern_type in self.patterns:
                match = re.search(pattern, text_lower, re.IGNORECASE)
                if match:
                    return self._parse_match(match, pattern_type, text)
        except Exception as e:
            self.logger.error(f"Error parsing salary '{text}': {e}")
        
        return None
    
    def _parse_match(self, match, pattern_type: str, original_text: str) -> Compensation:
        """Parse regex match into Compensation object."""
        try:
            if pattern_type == 'lpa_range':
                min_val = float(match.group(1))
                max_val = float(match.group(2))
                return Compensation(
                    salary_min=int(min_val * 100000),
                    salary_max=int(max_val * 100000),
                    currency='INR',
                    period='annual',
                    raw_text=match.group(0),
                    confidence=0.9
                )
            
            elif pattern_type == 'lpa_single':
                val = float(match.group(1))
                return Compensation(
                    salary_min=int(val * 100000),
                    salary_max=int(val * 100000),
                    currency='INR',
                    period='annual',
                    raw_text=match.group(0),
                    confidence=0.85
                )
            
            elif pattern_type == 'monthly':
                val = float(match.group(1))
                # Convert to annual
                annual = val * 1000 * 12 if val < 1000 else val * 12
                return Compensation(
                    salary_min=int(annual),
                    salary_max=int(annual),
                    currency='INR',
                    period='annual',
                    raw_text=match.group(0),
                    confidence=0.8
                )
            
            elif pattern_type in ['ctc', 'package_range']:
                val = float(match.group(1))
                return Compensation(
                    salary_min=int(val * 100000),
                    salary_max=int(val * 100000),
                    currency='INR',
                    period='annual',
                    raw_text=match.group(0),
                    confidence=0.85
                )
                
        except Exception as e:
            self.logger.error(f"Error parsing match: {e}")
        
        return None


# ===================================================================
# EXPERIENCE PARSER
# ===================================================================

class ExperienceParser:
    """
    Parse experience requirements from text.
    Handles ranges, single values, and descriptive text.
    """
    
    def __init__(self):
        """Initialize experience parser."""
        self.logger = logging.getLogger("ExperienceParser")
        
        self.patterns = [
            # Pattern: "2-5 years"
            r'(\d+)\s*(?:-|to)\s*(\d+)\s*years?',
            # Pattern: "2 years"
            r'(\d+)\s*years?\s+(?:of\s+)?experience',
            # Pattern: "0-2 years" (fresher)
            r'0\s*(?:-|to)\s*(\d+)\s*years?'
        ]
    
    def parse(self, text: str) -> Tuple[int, int]:
        """
        Parse experience requirement.
        
        Args:
            text: Input text
            
        Returns:
            Tuple of (min_years, max_years)
        """
        if not text or not isinstance(text, str):
            return (0, 0)
        
        text_lower = text.lower()
        
        # Check for fresher keywords
        if any(word in text_lower for word in ['fresher', 'freshers', 'entry level']):
            return (0, 0)
        
        try:
            for pattern in self.patterns:
                match = re.search(pattern, text_lower)
                if match:
                    groups = match.groups()
                    if len(groups) == 2:
                        return (int(groups[0]), int(groups[1]))
                    elif len(groups) == 1:
                        val = int(groups[0])
                        return (val, val)
        except Exception as e:
            self.logger.error(f"Error parsing experience '{text}': {e}")
        
        return (0, 0)


# ===================================================================
# DEADLINE PARSER
# ===================================================================

class DeadlineParser:
    """
    Parse and normalize deadline dates.
    Handles various date formats and relative dates.
    """
    
    def __init__(self):
        """Initialize deadline parser."""
        self.logger = logging.getLogger("DeadlineParser")
    
    def parse(self, text: str) -> Optional[str]:
        """
        Parse deadline from text.
        
        Args:
            text: Input text
            
        Returns:
            ISO format date string or None
        """
        if not text or not isinstance(text, str):
            return None
        
        try:
            # Try standard date formats
            date_patterns = [
                (r'(\d{1,2})[-/](\d{1,2})[-/](\d{4})', '%d-%m-%Y'),
                (r'(\d{1,2})[-/](\d{1,2})[-/](\d{2})', '%d-%m-%y'),
                (r'(\d{4})[-/](\d{1,2})[-/](\d{1,2})', '%Y-%m-%d')
            ]
            
            for pattern, fmt in date_patterns:
                match = re.search(pattern, text)
                if match:
                    date_str = match.group(0)
                    try:
                        dt = datetime.strptime(date_str, fmt)
                        return dt.strftime('%Y-%m-%d')
                    except:
                        continue
            
            # Handle relative dates
            if 'today' in text.lower():
                return datetime.now().strftime('%Y-%m-%d')
            
            # Check for "in X days"
            match = re.search(r'in\s+(\d+)\s+days?', text.lower())
            if match:
                days = int(match.group(1))
                future_date = datetime.now() + timedelta(days=days)
                return future_date.strftime('%Y-%m-%d')
                
        except Exception as e:
            self.logger.error(f"Error parsing deadline '{text}': {e}")
        
        return None


# ===================================================================
# RELATIONSHIP EXTRACTOR
# ===================================================================

class RelationshipExtractor:
    """
    Extract relationships between entities.
    Links companies with positions, skills, locations, etc.
    """
    
    def __init__(self):
        """Initialize relationship extractor."""
        self.logger = logging.getLogger("RelationshipExtractor")
        self.normalizer = EntityNormalizer()
        self.salary_parser = SalaryParser()
        self.experience_parser = ExperienceParser()
        self.deadline_parser = DeadlineParser()
    
    def extract_job_postings(
        self,
        row: pd.Series,
        email_id: str
    ) -> List[JobPosting]:
        """
        Extract structured job postings from a DataFrame row.
        
        Args:
            row: DataFrame row with extracted entities
            email_id: Unique email identifier
            
        Returns:
            List of structured JobPosting objects
        """
        job_postings = []
        
        try:
            # Parse entities from row
            companies = self._parse_list(row.get('companies_extracted', ''))
            positions = self._parse_list(row.get('positions_extracted', ''))
            skills = self._parse_list(row.get('skills_extracted', ''))
            locations = self._parse_list(row.get('locations_extracted', ''))
            salary_texts = self._parse_list(row.get('salary_info', ''))
            experience_texts = self._parse_list(row.get('experience_required', ''))
            degrees = self._parse_list(row.get('degrees_required', ''))
            
            # If no companies or positions, return empty
            if not companies or not positions:
                self.logger.debug(f"No companies or positions found in {email_id}")
                return []
            
            # Normalize entities
            skills = self.normalizer.normalize_skills_list(skills)
            degrees = self.normalizer.normalize_degrees_list(degrees)
            
            # Create job postings
            # Strategy: Create one posting per (company, position) pair
            for idx, (company_name, position_title) in enumerate(
                self._create_pairs(companies, positions)
            ):
                job_id = f"{email_id}_JOB_{idx+1}"
                
                try:
                    # Create structured entities
                    company = Company(
                        name=company_name,
                        canonical_name='',
                        confidence=0.85
                    )
                    
                    position = Position(
                        title=position_title,
                        confidence=0.85
                    )
                    
                    # Parse experience
                    exp_min, exp_max = 0, 0
                    if experience_texts:
                        exp_min, exp_max = self.experience_parser.parse(
                            experience_texts[0]
                        )
                    
                    requirements = Requirements(
                        skills=skills,
                        education=degrees,
                        experience_min=exp_min,
                        experience_max=exp_max
                    )
                    
                    # Parse location
                    location_city = locations[idx] if idx < len(locations) else (
                        locations[0] if locations else "Not Specified"
                    )
                    
                    location = Location(
                        city=location_city,
                        confidence=0.8
                    )
                    
                    # Parse salary
                    compensation = Compensation()
                    if salary_texts:
                        parsed_salary = self.salary_parser.parse(salary_texts[0])
                        if parsed_salary:
                            compensation = parsed_salary
                    
                    # Parse deadline
                    deadline = None
                    deadline_col = row.get('deadline', '')
                    if deadline_col:
                        deadline = self.deadline_parser.parse(str(deadline_col))
                    
                    application = Application(
                        deadline=deadline
                    )
                    
                    # Create job posting
                    job_posting = JobPosting(
                        job_id=job_id,
                        email_id=email_id,
                        company=company,
                        position=position,
                        requirements=requirements,
                        location=location,
                        compensation=compensation,
                        application=application,
                        metadata={
                            'extraction_timestamp': datetime.now().isoformat(),
                            'completeness_score': 0.0,
                            'source_subject': str(row.get('Subject', ''))[:100]
                        }
                    )
                    
                    # Calculate completeness
                    job_posting.metadata['completeness_score'] = (
                        job_posting.calculate_completeness()
                    )
                    
                    job_postings.append(job_posting)
                    
                except Exception as e:
                    self.logger.error(
                        f"Error creating job posting {job_id}: {e}"
                    )
                    continue
        
        except Exception as e:
            self.logger.error(
                f"Error extracting job postings from {email_id}: {e}"
            )
        
        return job_postings
    
    def _parse_list(self, text: str) -> List[str]:
        """Parse comma-separated string into list."""
        if not text or pd.isna(text) or text == '':
            return []
        return [item.strip() for item in str(text).split(',') if item.strip()]
    
    def _create_pairs(
        self,
        companies: List[str],
        positions: List[str]
    ) -> List[Tuple[str, str]]:
        """
        Create company-position pairs.
        
        Strategy:
        - If counts match: pair 1-to-1
        - If different: create all combinations (max 5)
        """
        if len(companies) == len(positions):
            return list(zip(companies, positions))
        
        # Create combinations but limit to avoid explosion
        pairs = []
        for company in companies[:3]:  # Max 3 companies
            for position in positions[:3]:  # Max 3 positions
                pairs.append((company, position))
        
        return pairs[:5]  # Max 5 job postings per email


# ===================================================================
# MAIN STRUCTURING PIPELINE
# ===================================================================

class EntityStructuringPipeline:
    """
    Main pipeline for entity structuring and relationship extraction.
    Orchestrates all components.
    """
    
    def __init__(self):
        """Initialize pipeline."""
        self.logger = logging.getLogger("StructuringPipeline")
        self.relationship_extractor = RelationshipExtractor()
        
        self.logger.info("="*70)
        self.logger.info("🏗️  ENTITY STRUCTURING PIPELINE INITIALIZED")
        self.logger.info("="*70)
    
    def process_dataset(
        self,
        input_csv: str,
        output_csv: str = "structured_job_postings.csv",
        output_json: str = "structured_job_postings.json"
    ) -> Tuple[pd.DataFrame, List[Dict]]:
        """
        Process entire dataset and create structured job postings.
        
        Args:
            input_csv: Input CSV file path
            output_csv: Output CSV file path
            output_json: Output JSON file path
            
        Returns:
            Tuple of (DataFrame, JSON list)
        """
        self.logger.info(f"\nLoading dataset: {input_csv}")
        
        try:
            df = pd.read_csv(input_csv)
            total_emails = len(df)
            self.logger.info(f"Loaded {total_emails} emails")
        except Exception as e:
            self.logger.error(f"Failed to load dataset: {e}")
            raise
        
        # Process each email
        self.logger.info(f"\nProcessing {total_emails} emails...")
        
        all_job_postings = []
        stats = {
            'total_emails': total_emails,
            'emails_with_jobs': 0,
            'total_jobs': 0,
            'avg_completeness': 0.0
        }
        
        for idx, row in df.iterrows():
            email_id = row.get('MessageId', f"EMAIL_{idx}")
            
            try:
                # Extract job postings from this email
                job_postings = self.relationship_extractor.extract_job_postings(
                    row, email_id
                )
                
                if job_postings:
                    stats['emails_with_jobs'] += 1
                    stats['total_jobs'] += len(job_postings)
                    
                    for job in job_postings:
                        all_job_postings.append(job.to_dict())
                
                # Log progress every 50 emails
                if (idx + 1) % 50 == 0:
                    self.logger.info(
                        f"   Processed {idx+1}/{total_emails} emails | "
                        f"Jobs extracted: {stats['total_jobs']}"
                    )
                    
            except Exception as e:
                self.logger.error(f"Error processing email {email_id}: {e}")
                continue
        
        # Calculate statistics
        if all_job_postings:
            completeness_scores = [
                job['metadata']['completeness_score'] 
                for job in all_job_postings
            ]
            stats['avg_completeness'] = np.mean(completeness_scores)
        
        # Log results
        self.logger.info(f"\n{'='*70}")
        self.logger.info(f"PROCESSING COMPLETE")
        self.logger.info(f"{'='*70}")
        self.logger.info(f"Total Emails Processed: {stats['total_emails']}")
        self.logger.info(
            f"Emails with Job Postings: {stats['emails_with_jobs']} "
            f"({stats['emails_with_jobs']/stats['total_emails']*100:.1f}%)"
        )
        self.logger.info(f"Total Job Postings Created: {stats['total_jobs']}")
        self.logger.info(
            f"Average Completeness Score: {stats['avg_completeness']:.2%}"
        )
        self.logger.info(f"{'='*70}\n")
        
        # Save outputs
        self._save_outputs(all_job_postings, output_csv, output_json)
        
        # Create DataFrame for easy querying
        jobs_df = self._create_dataframe(all_job_postings)
        
        return jobs_df, all_job_postings
    
    def _save_outputs(
        self,
        job_postings: List[Dict],
        csv_path: str,
        json_path: str
    ):
        """Save structured job postings to CSV and JSON."""
        try:
            # Save JSON (complete structure)
            self.logger.info(f"Saving JSON: {json_path}")
            with open(json_path, 'w', encoding='utf-8') as f:
                json.dump(job_postings, f, indent=2, ensure_ascii=False)
            self.logger.info(f"Saved {len(job_postings)} jobs to JSON")
            
            # Save CSV (flattened structure)
            self.logger.info(f"Saving CSV: {csv_path}")
            df = self._create_dataframe(job_postings)
            df.to_csv(csv_path, index=False)
            self.logger.info(f"Saved {len(df)} jobs to CSV\n")
            
        except Exception as e:
            self.logger.error(f"Error saving outputs: {e}")
    
    def _create_dataframe(self, job_postings: List[Dict]) -> pd.DataFrame:
        """
        Create flattened DataFrame from structured job postings.
        
        Args:
            job_postings: List of job posting dictionaries
            
        Returns:
            Flattened pandas DataFrame
        """
        flattened = []
        
        for job in job_postings:
            try:
                flat_job = {
                    # IDs
                    'job_id': job['job_id'],
                    'email_id': job['email_id'],
                    
                    # Company
                    'company_name': job['company']['name'],
                    'company_canonical': job['company']['canonical_name'],
                    'company_confidence': job['company']['confidence'],
                    
                    # Position
                    'position_title': job['position']['title'],
                    'position_level': job['position']['level'],
                    'position_confidence': job['position']['confidence'],
                    
                    # Requirements
                    'skills_required': ', '.join(job['requirements']['skills']),
                    'skills_count': len(job['requirements']['skills']),
                    'education_required': ', '.join(job['requirements']['education']),
                    'experience_min_years': job['requirements']['experience_min'],
                    'experience_max_years': job['requirements']['experience_max'],
                    'experience_type': job['requirements']['experience_type'],
                    
                    # Location
                    'location_city': job['location']['city'],
                    'location_state': job['location']['state'],
                    'work_mode': job['location']['work_mode'],
                    'location_confidence': job['location']['confidence'],
                    
                    # Compensation
                    'salary_min': job['compensation']['salary_min'],
                    'salary_max': job['compensation']['salary_max'],
                    'salary_currency': job['compensation']['currency'],
                    'salary_period': job['compensation']['period'],
                    'salary_raw_text': job['compensation']['raw_text'],
                    'salary_confidence': job['compensation']['confidence'],
                    
                    # Application
                    'application_deadline': job['application']['deadline'],
                    'apply_link': job['application']['apply_link'],
                    'contact_email': job['application']['contact_email'],
                    
                    # Metadata
                    'completeness_score': job['metadata']['completeness_score'],
                    'extraction_timestamp': job['metadata']['extraction_timestamp'],
                    'source_subject': job['metadata']['source_subject']
                }
                
                flattened.append(flat_job)
                
            except Exception as e:
                self.logger.error(f"Error flattening job {job.get('job_id')}: {e}")
                continue
        
        return pd.DataFrame(flattened)
    
    def generate_analytics(
        self,
        csv_path: str = "structured_job_postings.csv"
    ):
        """
        Generate analytics report on structured data.
        
        Args:
            csv_path: Path to structured CSV file
        """
        self.logger.info(f"\n{'='*70}")
        self.logger.info(f"ANALYTICS REPORT")
        self.logger.info(f"{'='*70}\n")
        
        try:
            df = pd.read_csv(csv_path)
            
            # Basic statistics
            self.logger.info(f"BASIC STATISTICS:")
            self.logger.info(f"   Total Job Postings: {len(df)}")
            self.logger.info(f"   Unique Companies: {df['company_name'].nunique()}")
            self.logger.info(f"   Unique Positions: {df['position_title'].nunique()}")
            self.logger.info(f"   Unique Locations: {df['location_city'].nunique()}")
            
            # Top companies
            self.logger.info(f"\nTOP 15 HIRING COMPANIES:")
            top_companies = df['company_name'].value_counts().head(15)
            for company, count in top_companies.items():
                self.logger.info(f"   {company}: {count} positions")
            
            # Top positions
            self.logger.info(f"\nTOP 15 JOB POSITIONS:")
            top_positions = df['position_title'].value_counts().head(15)
            for position, count in top_positions.items():
                self.logger.info(f"   {position}: {count} openings")
            
            # Top locations
            self.logger.info(f"\nTOP 10 LOCATIONS:")
            top_locations = df['location_city'].value_counts().head(10)
            for location, count in top_locations.items():
                self.logger.info(f"   {location}: {count} jobs")
            
            # Top skills
            all_skills = []
            for skills_str in df['skills_required']:
                if pd.notna(skills_str) and skills_str:
                    all_skills.extend([s.strip() for s in skills_str.split(',')])
            
            if all_skills:
                skill_counts = Counter(all_skills)
                self.logger.info(f"\nTOP 20 MOST DEMANDED SKILLS:")
                for skill, count in skill_counts.most_common(20):
                    self.logger.info(f"   {skill}: {count} mentions")
            
            # Salary statistics
            salary_data = df[df['salary_max'] > 0]['salary_max']
            if len(salary_data) > 0:
                self.logger.info(f"\nSALARY STATISTICS:")
                self.logger.info(
                    f"   Jobs with Salary Info: {len(salary_data)} "
                    f"({len(salary_data)/len(df)*100:.1f}%)"
                )
                self.logger.info(
                    f"   Average Salary: ₹{salary_data.mean()/100000:.2f} LPA"
                )
                self.logger.info(
                    f"   Median Salary: ₹{salary_data.median()/100000:.2f} LPA"
                )
                self.logger.info(
                    f"   Min Salary: ₹{salary_data.min()/100000:.2f} LPA"
                )
                self.logger.info(
                    f"   Max Salary: ₹{salary_data.max()/100000:.2f} LPA"
                )
            
            # Experience distribution
            self.logger.info(f"\nEXPERIENCE LEVEL DISTRIBUTION:")
            exp_dist = df['experience_type'].value_counts()
            for exp_type, count in exp_dist.items():
                self.logger.info(
                    f"   {exp_type}: {count} ({count/len(df)*100:.1f}%)"
                )
            
            # Work mode distribution
            self.logger.info(f"\nWORK MODE DISTRIBUTION:")
            work_mode_dist = df['work_mode'].value_counts()
            for mode, count in work_mode_dist.items():
                self.logger.info(
                    f"   {mode}: {count} ({count/len(df)*100:.1f}%)"
                )
            
            # Completeness distribution
            self.logger.info(f"\nDATA COMPLETENESS:")
            avg_completeness = df['completeness_score'].mean()
            self.logger.info(f"   Average Completeness: {avg_completeness:.2%}")
            
            high_quality = len(df[df['completeness_score'] >= 0.7])
            medium_quality = len(
                df[(df['completeness_score'] >= 0.5) & 
                   (df['completeness_score'] < 0.7)]
            )
            low_quality = len(df[df['completeness_score'] < 0.5])
            
            self.logger.info(
                f"   High Quality (≥70%): {high_quality} "
                f"({high_quality/len(df)*100:.1f}%)"
            )
            self.logger.info(
                f"   Medium Quality (50-70%): {medium_quality} "
                f"({medium_quality/len(df)*100:.1f}%)"
            )
            self.logger.info(
                f"   Low Quality (<50%): {low_quality} "
                f"({low_quality/len(df)*100:.1f}%)"
            )
            
            self.logger.info(f"\n{'='*70}\n")
            
        except Exception as e:
            self.logger.error(f"Error generating analytics: {e}")


# ===================================================================
# MAIN EXECUTION
# ===================================================================

def main():
    """Main execution function."""
    
    logger.info("\n" + "="*70)
    logger.info("STARTING ENTITY STRUCTURING PIPELINE")
    logger.info("="*70 + "\n")
    
    try:
        # Initialize pipeline
        pipeline = EntityStructuringPipeline()
        
        # Process dataset
        input_path = r"D:\Projects By Month\November 2025\Placement Mail Analysis System\.venv\Phase_scripts\Phase 2\relevant_placement_emails.csv"
        input_file = input_path
        output_csv = "structured_job_postings.csv"
        output_json = "structured_job_postings.json"
        
        jobs_df, jobs_list = pipeline.process_dataset(
            input_csv=input_file,
            output_csv=output_csv,
            output_json=output_json
        )
        
        # Generate analytics
        pipeline.generate_analytics(output_csv)
        
        logger.info("="*70)
        logger.info("PIPELINE COMPLETED SUCCESSFULLY")
        logger.info("="*70)
        logger.info(f"Output Files:")
        logger.info(f"   - CSV: {output_csv}")
        logger.info(f"   - JSON: {output_json}")
        logger.info(f"   - Log: entity_structuring.log")
        logger.info("="*70 + "\n")
        
        return jobs_df, jobs_list
        
    except Exception as e:
        logger.error(f"PIPELINE FAILED: {e}")
        raise


if __name__ == "__main__":
    # Execute pipeline
    jobs_df, jobs_json = main()
    
    # Optional: Display sample
    print("\nSAMPLE STRUCTURED JOB POSTING:")
    print("="*70)
    if len(jobs_df) > 0:
        sample = jobs_df.iloc[0].to_dict()
        for key, value in sample.items():
            print(f"{key}: {value}")
    print("="*70)